In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab_Notebooks/CSC8639/fewshot_foodseg103')

In [3]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unin

In [4]:
# Train
# FoodSeg103

args = [
    '--split_class_json', '../data/foodseg103/class_split.json',
    '--class2images_mapping_json', '../data/foodseg103/class2images_mapping.json',
    '--img_dir', '../data/foodseg103/rawdata/image',
    '--mask_dir', '../data/foodseg103/rawdata/mask',

    '--model_name', 'resnet50_8x_cosine_accu_bgmix_scaler',  # 选择模型时更改
    '--seed', '42',
    '--beta_mix', '1',
    '--alpha_bg', '1',

    '--n_way', '1',
    '--k_shot', '5',
    '--q_query', '5',
    '--train_episodes', '100',
    '--val_episodes', '50',

    '--batch_size', '1',
    '--num_workers', '10',
    '--epochs', '50',

    '--optimizer', 'adamw',  # adam/adamw/sgd
    '--scheduler', 'cosine',  # step/cosine/cosine_restart/plateau
    '--learning_rate', '1e-3',
    '--weight_decay', '1e-4',
    '--loss', 'ce',  # ce/dice/focal/combo
    # '--bg_weight', '0.8',  # loss中背景类别权重（已在trainshell中禁用）
    '--metric', 'miou', 'dice', 'precision', 'recall', 'acc',  # 启用的metrics

    '--main_metric', 'miou',  # loss/miou/dice/acc  # 指定监控指标（默认为loss），最佳模型保存策略、plateau学习率下降策略、早停策略共用此指标
    # '--early_stopping_patience', '25',  # 设置早停轮数即启用早停

    # '--weight_path',  # 预训练模型/上次训练模型 路径
    # '--is_resume',  # 断点训练
    '--use_amp',

    '--save_dir', 'result_foodseg103/resnet50_8x_cosine_accu_bgmix1_scaler1',  # 每次更改，与 wandb 项目名相同
    '--wandb'  # 设置即启用
    ]

!python train_shell.py {' '.join(args)}

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ltc735983415 (ltc735983415-newcastle-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /content/drive/.shortcut-targets-by-id/1fuvwH2wo3q3zusv_klINh1muke4KOxCQ/Colab_Notebooks/CSC8639/fewshot_foodseg103/wandb/run-20250730_003809-ylp1bv2j
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run resnet50_8x_cosine_accu_bgmix1_scaler1
wandb: ⭐️ View project at https://wandb.ai/ltc735983415-newcastle-university/CSC8639_FewShot_FoodSeg103
wandb: 🚀 View run at https://wandb.ai/ltc735983415-newcastle-university/CSC8639_FewShot_FoodSeg103/runs/ylp1bv2j
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:00<00:00, 224MB/s]
Start tr

In [6]:
# Predict
# FoodSeg103

args = [
    '--split_class_json', '../data/foodseg103/class_split.json',
    '--class2images_mapping_json', '../data/foodseg103/class2images_mapping.json',
    '--img_dir', '../data/foodseg103/rawdata/image',
    '--mask_dir', '../data/foodseg103/rawdata/mask',

    '--model_name', 'resnet50_8x_cosine_accu_bgmix_scaler',  # 选择模型时更改
    '--seed', '42',
    '--beta_mix', '1',
    '--alpha_bg', '1',

    '--n_way', '1',
    '--k_shot', '5',
    '--q_query', '5',
    '--pred_episodes', '100',

    '--batch_size', '1',
    '--num_workers', '10',
    '--metric', 'miou', 'dice', 'precision', 'recall', 'acc',

    '--weight_path', 'result_foodseg103/resnet50_8x_cosine_accu_bgmix1_scaler1/checkpoint/model_best.pth',  # 每次更改
    '--use_amp',

    '--save_dir', 'result_foodseg103/resnet50_8x_cosine_accu_bgmix1_scaler1/predict',  # 每次更改，与 wandb 项目名相同
    '--wandb',  # 设置即启用
]

!python predict_shell.py {' '.join(args)}

^C
